In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import duckdb
from uuid import uuid4
conn = duckdb.connect()

In [3]:
query = """
CREATE TABLE IF NOT EXISTS conversation (
    convo_id VARCHAR(36) PRIMARY KEY,
    content TEXT NOT NULL,
    role VARCHAR(10) CHECK (role IN ('user', 'assistant')),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    session_id VARCHAR(36)
);
"""
conn.execute(query).fetchall()

[]

In [4]:
conn.execute("SHOW TABLES;").df()

,name
0,conversation


In [5]:
query = """
INSERT INTO conversation (convo_id, content, role) VALUES (?, ?, ?);
"""

conn.execute(query, [str(uuid4()), "Hello, world!", "user"]).fetchall()

[(1,)]

In [6]:
query = """
SELECT * FROM conversation;
"""
conn.execute(query).df()

,convo_id,content,role,created_at,updated_at,session_id
0,277bd8ec-39cd-4c38-b860-be35894b31f1,"Hello, world!",user,2025-11-15 12:19:10.790612,2025-11-15 12:19:10.790612,None


In [7]:
convo_id = conn.execute("SELECT convo_id FROM conversation LIMIT 1;").fetchone()[0]
query = """
UPDATE conversation
SET content = ?,
updated_at = CURRENT_TIMESTAMP
WHERE convo_id = ?;
"""

conn.execute(query, ["Holla world!", convo_id]).fetchall()

[(1,)]

In [8]:
query = """
SELECT * FROM conversation;
"""
conn.execute(query).df()

,convo_id,content,role,created_at,updated_at,session_id
0,277bd8ec-39cd-4c38-b860-be35894b31f1,Holla world!,user,2025-11-15 12:19:10.790612,2025-11-15 12:19:12.725026,None


In [9]:
query = """
DELETE FROM conversation WHERE convo_id = ?;
"""

conn.execute(query, [convo_id]).fetchall()

[(1,)]

In [10]:
query = """
SELECT * FROM conversation;
"""
conn.execute(query).df()

,convo_id,content,role,created_at,updated_at,session_id


In [11]:
from uuid import uuid4
from datetime import datetime
from pydantic import BaseModel, Field
from typing import List, Optional, Any, Optional, Dict, Literal
from enum import StrEnum

class ConversationRole(StrEnum):
    USER = "user"
    ASSISTANT = "assistant"

class Artifact(BaseModel):
    type:Literal["sql", "data", "html"]
    data:Any

class Conversation(BaseModel):
    conversation_id:str = Field(default_factory=lambda: str(uuid4()))
    content:str
    role:ConversationRole
    created_at:datetime = Field(default_factory=datetime.now)
    updated_at:datetime = Field(default_factory=datetime.now)
    artifacts:Optional[List[Artifact]] = Field(default=None)
    session_id:Optional[str] = Field(default=None)

Conversation(content="Hi", role=ConversationRole.USER)

Conversation(conversation_id='7c375e64-19d4-4396-89c2-e9129d2cdaeb', content='Hi', role=<ConversationRole.USER: 'user'>, created_at=datetime.datetime(2025, 11, 15, 12, 19, 58, 18839), updated_at=datetime.datetime(2025, 11, 15, 12, 19, 58, 18839), artifacts=None, session_id=None)

In [14]:
from enum import Enum

class Status(Enum):
    PENDING = "pending" 
    COMPLETED = "completed"

# These are PREVENTED:
# Status.PENDING = "oops"  # Error!
# Status.INVALID = "hack"  # Error!

# Built-in validation:
Status("pending")  # Returns Status.PENDING
# Status("invalid")  # Raises ValueError

# Iteration works:
for status in Status:
    print(status)  # All valid values


Status.PENDING
Status.COMPLETED


In [19]:
from enum import Enum

class DatabaseType(Enum):
    SQLITE = "sqlite"
    POSTGRES = "postgresql"
    MYSQL = "mysql"

class LogLevel(Enum):
    DEBUG = "debug"
    INFO = "info"
    ERROR = "error"

# Simple validation function
def validate_enum(value, enum_class):
    try:
        return enum_class(value)
    except ValueError:
        raise ValueError(f"Invalid {enum_class.__name__}: {value}")

# Usage
db_type = validate_enum("sqlite", DatabaseType)  # Returns DatabaseType.SQLITE
log_level = validate_enum("info", LogLevel)      # Returns LogLevel.INFO


In [22]:
LogLevel("error")

<LogLevel.ERROR: 'error'>